In [ ]:
from collections import defaultdict
import datetime as dt
import os, random

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


from research.weight_estimation.gtsf_data.gtsf_dataset import GTSFDataset, BODY_PARTS


In [ ]:
akpd_scorer_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/keypoint-detection-scorer/akpd_scorer_model_TF.h5'
gtsf_dataset = GTSFDataset('2019-02-01', '2020-03-30', akpd_scorer_url)
df = gtsf_dataset.get_prepared_dataset()

<h1> Construct Point Cloud Data Transform </h1>

In [ ]:
def jitter_wkps(wkps, cm, base_jitter):
    wkps_jittered = []
    for idx in range(len(BODY_PARTS)):
        x_p_left = wkps[idx, 0] * cm['focalLengthPixel'] / wkps[idx, 1]
        y_p_left = wkps[idx, 2] * cm['focalLengthPixel'] / wkps[idx, 1]
        disparity = cm['focalLengthPixel'] * cm['baseline'] / wkps[idx, 1]
        x_p_left_jitter = np.random.normal(0, base_jitter)
        x_p_right_jitter = np.random.normal(0, base_jitter)
        disparity_jitter = x_p_left_jitter + x_p_right_jitter
        
        x_p_left_jittered = x_p_left + x_p_left_jitter
        disparity_jittered = disparity + disparity_jitter
        depth_jittered = cm['focalLengthPixel'] * cm['baseline'] / disparity_jittered
        x_jittered = x_p_left_jittered * depth_jittered / cm['focalLengthPixel']
        y_jittered = y_p_left * depth_jittered / cm['focalLengthPixel']
        wkp_jittered = [x_jittered, depth_jittered, y_jittered]
        wkps_jittered.append(wkp_jittered)
    if wkps.shape[0] == len(BODY_PARTS) + 1:
        wkps_jittered.append(wkps[len(BODY_PARTS), :].tolist())
    wkps_jittered = np.array(wkps_jittered)
    return wkps_jittered


def get_jittered_keypoints(wkps, cm, base_jitter=5):    
    # put at random depth and apply jitter
    depth = np.random.uniform(low=0.3, high=2.5)
    wkps[:, 1] = wkps[:, 1] - np.median(wkps[:, 1]) + depth
    
    # apply jitter
    jittered_wkps = jitter_wkps(wkps, cm, base_jitter)
    return jittered_wkps
        


<h1> Train Neural Network </h1>

In [ ]:
class KeypointsDataset(Dataset):
    """Keypoints dataset
    This is the base version of the dataset that is used to map 3D keypoints to a
    biomass estimate. The label is the weight, and the input is the 3D workd keypoints
    obtained during triangulation
    """

    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform


    def __len__(self):
        return self.df.shape[0]


    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        if self.transform:
            input_sample = {
                'kp_input': row.centered_keypoint_arr,
                'cm': row.camera_metadata,
                'stereo_pair_id': row.id,
            }
            if 'weight' in dict(row).keys():
                input_sample['label'] = row.weight
            sample = self.transform(input_sample)
            return sample

        world_keypoints = row.world_keypoints
        weight = row.weight

        sample = {'kp_input': world_keypoints, 'label': weight, 'stereo_pair_id': row.id}

        return sample

class NormalizedCentered3D(object):
    
    def __init__(self, base_jitter):
        self.base_jitter = base_jitter

    def __call__(self, sample):
        keypoint_arr, cm, stereo_pair_id, label = \
            sample['kp_input'], sample['cm'], sample.get('stereo_pair_id'), sample.get('label')
    
        jittered_wkps = get_jittered_keypoints(keypoint_arr, cm, base_jitter=self.base_jitter)
        normalized_label = label * 1e-4
        
        transformed_sample = {
            'kp_input': jittered_wkps,
            'label': normalized_label,
            'stereo_pair_id': stereo_pair_id,
            'cm': cm,
            'single_point_inference': sample.get('single_point_inference')
        }

        return transformed_sample
    
class ToTensor(object):
    
    def __call__(self, sample):
        x, label, stereo_pair_id = \
            sample['kp_input'], sample.get('label'), sample.get('stereo_pair_id')
        
        if sample.get('single_point_inference'):
            x = np.array([x])
        else:
            x = np.array(x)
        
        kp_input_tensor = torch.from_numpy(x).float()
        
        tensorized_sample = {
            'kp_input': kp_input_tensor
        }

        if label:
            label_tensor = torch.from_numpy(np.array([label])).float() if label else None
            tensorized_sample['label'] = label_tensor

        if stereo_pair_id:
            tensorized_sample['stereo_pair_id'] = stereo_pair_id

        
        return tensorized_sample
        

<h1> Define train and test data loaders </h1>

In [ ]:
gtsf_fish_identifiers = list(df.fish_id.unique())
train_size = int(0.8 * len(gtsf_fish_identifiers))
fish_ids = random.sample(gtsf_fish_identifiers, train_size)
date_mask = (df.captured_at < '2019-09-10')
train_mask = date_mask & df.fish_id.isin(fish_ids)
test_mask = date_mask & ~df.fish_id.isin(fish_ids)

train_dataset = KeypointsDataset(df[train_mask], transform=transforms.Compose([
                                                  NormalizedCentered3D(10),
                                                  ToTensor()
                                              ]))

train_dataloader = DataLoader(train_dataset, batch_size=25, shuffle=True, num_workers=1)

test_dataset = KeypointsDataset(df[test_mask], transform=transforms.Compose([
                                                      NormalizedCentered3D(10),
                                                      ToTensor()
                                                  ]))

test_dataloader = DataLoader(test_dataset, batch_size=25, shuffle=True, num_workers=1)

In [ ]:
for data in train_dataloader:
    new_wkps = data['kp_input']
    break

In [ ]:
%matplotlib inline
plt.figure(figsize=(20, 10))
plt.scatter(new_wkps[0][:, 0], new_wkps[0][:, 2], color='red')
plt.show()

In [ ]:
# TODO: Define your network architecture here
import torch
from torch import nn

class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(24, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.output(x)
        return x
        


In [ ]:
run_name = 'batch_25_jitter_10_lr_1e-4_v2_rot_fix'
write_outputs = True

# establish output directory where model .pb files will be written
if write_outputs:
    dt_now = dt.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    output_base = '/root/data/alok/biomass_estimation/results/neural_network'
    output_dir = os.path.join(output_base, run_name, dt_now)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

# instantiate neural network
network = Network()
epochs = 1000
optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)
criterion = torch.nn.MSELoss()

# track train and test losses
train_losses, test_losses = [], []

seed = 0
for epoch in range(epochs):
    network.train()
    np.random.seed(seed)
    seed += 1
    running_loss = 0.0
    for i, data_batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        X_batch, y_batch, kpid_batch = \
            data_batch['kp_input'], data_batch['label'], data_batch['stereo_pair_id']
        y_pred = network(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i > 0 and i % 100 == 0:
            print(running_loss / i)
            
    # run on test set
    else:
        test_running_loss = 0.0
        with torch.no_grad():
            network.eval()
            for i, data_batch in enumerate(test_dataloader):
                X_batch, y_batch, kpid_batch = \
                    data_batch['kp_input'], data_batch['label'], data_batch['stereo_pair_id']
                y_pred = network(X_batch)
                loss = criterion(y_pred, y_batch)
                test_running_loss += loss.item()

    train_loss_for_epoch = running_loss / len(train_dataloader)
    test_loss_for_epoch = test_running_loss / len(test_dataloader)
    train_losses.append(train_loss_for_epoch)
    test_losses.append(test_loss_for_epoch)
    
    # save current state of network
    if write_outputs:
        f_name = 'nn_epoch_{}.pb'.format(str(epoch).zfill(3))
        f_path = os.path.join(output_dir, f_name)
        torch.save(network, f_path)
    
    # print current loss values
    print('-'*20)
    print('Epoch: {}'.format(epoch))
    print('Train Loss: {}'.format(train_loss_for_epoch))
    print('Test Loss: {}'.format(test_loss_for_epoch))
    
    


In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(range(len(train_losses)), train_losses, color='blue', label='training loss')
plt.plot(range(len(test_losses)), test_losses, color='orange', label='validation loss')
plt.ylim([0, 0.01])
plt.xlabel('Epoch')
plt.ylabel('Loss value (MSE)')
plt.title('Loss curves (MSE - Adam optimizer)')
plt.legend()
plt.grid()
plt.show()

In [ ]:
oos_dataset = KeypointsDataset(df[train_mask], transform=transforms.Compose([
                                                  NormalizedCentered3D(0),
                                                  ToTensor()
                                              ]))

oos_dataloader = DataLoader(oos_dataset, batch_size=25, shuffle=True, num_workers=1)

<h1> Display Accuracy Numbers with Best Model </h1>

In [ ]:
best_epoch = np.argmin(test_losses)
best_network = torch.load(os.path.join(output_dir, f'nn_epoch_{best_epoch}.pb'))

with torch.no_grad():
    best_network.eval()
    y_preds_train, y_gt_train, y_preds_test, y_gt_test, kpids_train, kpids_test = \
        [], [], [], [], [], []
    for i, data_batch in enumerate(oos_dataloader):
        optimizer.zero_grad()
        X_batch, y_batch, kpid_batch = \
            data_batch['kp_input'], data_batch['label'], data_batch['stereo_pair_id']
        y_pred = best_network(X_batch)
        y_preds_train.extend(list(y_pred.numpy().flatten()))
        y_gt_train.extend(list(y_batch.numpy().flatten()))
        kpids_train.extend(list(kpid_batch.numpy().flatten()))
    
    for i, data_batch in enumerate(test_dataloader):
        optimizer.zero_grad()
        X_batch, y_batch, kpid_batch = \
            data_batch['kp_input'], data_batch['label'], data_batch['stereo_pair_id']
        y_pred = best_network(X_batch)
        y_preds_test.extend(list(y_pred.numpy().flatten()))
        y_gt_test.extend(list(y_batch.numpy().flatten()))
        kpids_test.extend(list(kpid_batch.numpy().flatten()))

analysis_df_train = pd.DataFrame({'y_pred': 1e4 * np.array(y_preds_train), 
                                  'y_gt': 1e4 * np.array(y_gt_train), 
                                  'kpid': kpids_train})
analysis_df_test = pd.DataFrame({'y_pred': 1e4 * np.array(y_preds_test), 
                                 'y_gt': 1e4 * np.array(y_gt_test), 
                                 'kpid': kpids_test})

In [ ]:
mean_absolute_err_pct = np.mean(np.abs(((analysis_df_train.y_pred - analysis_df_train.y_gt) / analysis_df_train.y_gt).values))
print(f'Mean absolute error percentage (train): {round(100 * mean_absolute_err_pct, 2)}%')
mean_absolute_err_pct = np.mean(np.abs(((analysis_df_test.y_pred - analysis_df_test.y_gt) / analysis_df_test.y_gt).values))
print(f'Mean absolute error percentage (test): {round(100 * mean_absolute_err_pct, 2)}%')

median_absolute_err_pct = np.median(np.abs(((analysis_df_train.y_pred - analysis_df_train.y_gt) / analysis_df_train.y_gt).values))
print(f'Median absolute error percentage (train): {round(100 * median_absolute_err_pct, 2)}%')
median_absolute_err_pct = np.median(np.abs(((analysis_df_test.y_pred - analysis_df_test.y_gt) / analysis_df_test.y_gt).values))
print(f'Median absolute error percentage (test): {round(100 * median_absolute_err_pct, 2)}%')

In [ ]:
(analysis_df_train.y_pred.mean() - analysis_df_train.y_gt.mean()) / analysis_df_train.y_gt.mean()

<h1> Show Error with respect to K-Factor </h1>

In [ ]:
df['k_factor'] = 1e5 * df.weight / df.data.apply(lambda x: x['lengthMms']**3).astype(float)
analysis_df_train['error_raw'] = analysis_df_train.y_pred - analysis_df_train.y_gt
analysis_df_train['error_pct'] = analysis_df_train.error_raw / analysis_df_train.y_gt
analysis_df_train['abs_error_pct'] = (analysis_df_train.error_raw / analysis_df_train.y_gt).abs()
fish_ids, kfs = [], []
for idx, row in analysis_df_train.iterrows():
    kpid_mask = df.id == row.kpid
    fish_id = df[kpid_mask].fish_id.iloc[0]
    kf = df[kpid_mask].k_factor.iloc[0]
    fish_ids.append(fish_id)
    kfs.append(kf)
analysis_df_train['fish_id'] = fish_ids
analysis_df_train['kf'] = kfs
    
fish_analysis_data = defaultdict(list)
for fish_id in sorted(analysis_df_train.fish_id.unique()):
    mask = analysis_df_train.fish_id == fish_id
    fish_analysis_data['fish_id'].append(fish_id)
    fish_analysis_data['num_stereo_images'].append(analysis_df_train[mask].shape[0])
    fish_analysis_data['mean_err_pct'].append(analysis_df_train[mask].error_pct.mean())
    fish_analysis_data['std_err_pct'].append(analysis_df_train[mask].error_pct.std())
    fish_analysis_data['kf'].append(analysis_df_train[mask].kf.iloc[0])
    
fish_analysis_df = pd.DataFrame(fish_analysis_data)

In [ ]:
plt.figure(figsize=(20, 10))
plt.scatter(fish_analysis_df[fish_analysis_df.num_stereo_images > 20].kf.values, 
            fish_analysis_df[fish_analysis_df.num_stereo_images > 20].mean_err_pct.values)
plt.grid()
plt.show()

In [ ]:
with torch.no_grad():
    network.eval()
    y_preds_train, y_gt_train, y_preds_test, y_gt_test = [], [], [], []
    for i, data_batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        X_batch, y_batch, kpid_batch = \
            data_batch['kp_input'], data_batch['label'], data_batch['stereo_pair_id']
        y_pred = network(X_batch)
        y_preds_train.extend(list(y_pred.numpy().flatten()))
        y_gt_train.extend(list(y_batch.numpy().flatten()))
    
    for i, data_batch in enumerate(train_dataloader):
        optimizer.zero_grad()
        X_batch, y_batch, kpid_batch = \
            data_batch['kp_input'], data_batch['label'], data_batch['stereo_pair_id']
        y_pred = network(X_batch)
        y_preds_test.extend(list(y_pred.numpy().flatten()))
        y_gt_test.extend(list(y_batch.numpy().flatten()))


In [ ]:
y_preds_test = 1e4 * np.array(y_preds_test)
y_gt_test = 1e4 * np.array(y_gt_test)

In [ ]:
np.median(np.abs((y_preds_test - y_gt_test) / y_gt_test))

In [ ]:
plt.figure(figsize=(20, 10))
plt.scatter(1e4*np.array(y_gt_train), 1e4*np.array(y_preds_train), color='blue', alpha=1.0)
plt.scatter(1e4*np.array(y_gt_test), 1e4*np.array(y_preds_test), color='red', alpha=0.3)
plt.plot([0, 10000], [0, 10000], color='blue')
plt.grid()
plt.show()

In [ ]:
kf_values = np.arange(0.7, 1.8, 0.1)
sample_sizes = []
abs_err_list = []
mean_err_list = []
for idx in range(len(kf_values) - 1):
    mask = (analysis_df_train.kf > kf_values[idx]) & (analysis_df_train.kf < kf_values[idx + 1])
    sample_sizes.append(mask.sum())
    abs_err = np.mean(np.abs((analysis_df_train[mask].y_pred - analysis_df_train[mask].y_gt) / analysis_df_train[mask].y_gt))
    abs_err_list.append(abs_err)
    mean_err = np.mean((analysis_df_train[mask].y_pred - analysis_df_train[mask].y_gt) / analysis_df_train[mask].y_gt)
    mean_err_list.append(mean_err)

pd.DataFrame({'sample_size': sample_sizes, 'mean_err': mean_err_list, 'abs_err': abs_err_list})

In [ ]:
sorted_urls = list(df.sort_values('k_factor', ascending=False).left_url.values)

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
image_f, _, _ = s3_access_utils.download_from_url(sorted_urls[8])
im = Image.open(image_f)
im

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')
image_f, _, _ = s3_access_utils.download_from_url(sorted_urls[-3])
im = Image.open(image_f)
im

In [ ]:
torch.save(best_network.state_dict(), '/root/data/alok/biomass_estimation/playground/nn_8_keypoints_jitter_10.pb')

In [ ]:
new_model = Network()
new_model.load_state_dict(torch.load('/root/data/alok/biomass_estimation/playground/nn_8_keypoints_jitter_10.pb'))

In [ ]:
new_model(torch.from_numpy(np.array([df.keypoint_arr.iloc[0]])).float())

In [ ]:
# instantiate weight estimator class
model_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-03-26T11-58-00/nn_8_keypoints_jitter_10.pb'
s3_access_utils = S3AccessUtils('/root/data')
model_f, _, _ = s3_access_utils.download_from_url(model_url)
weight_estimator = WeightEstimator(model_f)

# generate sample predictions
weights = []
for idx, row in df.iterrows():
    keypoints, camera_metadata = row.keypoints, row.camera_metadata
    weight_prediction = weight_estimator.predict(keypoints, camera_metadata)
    weights.append(weight_prediction)
    if len(weights) % 1000 == 0:
        print(len(weights))


In [ ]:
df.weight.mean()

In [ ]:
normalized_centered_3d = NormalizedCentered3D(0)
to_tensor = ToTensor()
preds = []
for idx, row in df.iterrows():
    input_sample = {
        'kp_input': get_keypoint_arr(row.keypoints, row.camera_metadata),
        'cm': row.camera_metadata,
        'stereo_pair_id': row.id,
        'label': row.weight,
        'single_point_inference': True
    }
    kps = normalized_centered_3d.__call__(input_sample)
    kps_tensor = to_tensor(kps)
    pred = 1e4 * best_network(kps_tensor['kp_input']).item()
    preds.append(pred)

In [ ]:
for data in train_dataloader:
    new_wkps = data['kp_input']
    print(data['stereo_pair_id'])
    break

In [ ]:
new_wkps[0]